In [1]:
from tr_kfac_opt import KFACOptimizer
%load_ext autoreload
%autoreload 2

In [1]:
import ray
ray.init(num_cpus=50, num_gpus=1)

2020-11-08 16:47:10,423	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '128.10.130.25',
 'raylet_ip_address': '128.10.130.25',
 'redis_address': '128.10.130.25:6379',
 'object_store_address': '/tmp/ray/session_2020-11-08_16-47-09_303808_6164/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-08_16-47-09_303808_6164/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-11-08_16-47-09_303808_6164',
 'metrics_export_port': 50794}

In [2]:
import CoRec
import gym
import numpy as np

import stable_baselines3 as sb3
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from ray import tune

In [3]:
class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)

    def reward(self, r):
        if r <= -1:
            r = 0
        else:
            r += 0.1

        return r

In [ ]:
def ppo_hyper(**kwargs): return kwargs

config = ppo_hyper(
    learning_rate=tune.loguniform(1e-6, 0.01),
    n_steps=tune.choice([256, 512, 1024, 2048, 4096]),
    batch_size=tune.choice([32, 64, 128, 256, 512, 1024, 2048]),
    n_epochs=tune.randint(4, 32),
    gamma=tune.uniform(0.95, 1),
    gae_lambda=tune.uniform(0.95, 0.999),
    clip_range=tune.uniform(0.01, 0.5),
    clip_range_vf=None,
    ent_coef=0.05,
    vf_coef=tune.uniform(0.2, 0.8),
    max_grad_norm=0.5,
    use_sde=False,
    sde_sample_freq=-1,
    target_kl=None,
    seed=1
)
    
config

In [5]:
def env_func(): return RewardWrapper(gym.make("F16GCAS-v3"))

env = make_vec_env(env_func, n_envs=4)
model = PPO(MlpPolicy, env, verbose=1)

## Only change is here
see <--

In [ ]:
class ReportCallback(sb3.common.callbacks.BaseCallback):
    def __init__(self, verbose=0):
        super(ReportCallback, self).__init__(verbose)
        
    def _on_rollout_end(self) -> None:
        ep_rewards = [ep_info["r"] for ep_info in self.model.ep_info_buffer]
        ep_rew_mean = np.mean(ep_rewards)
        tune.report(ep_rew = ep_rew_mean)
    
    def _on_step(self) -> bool:
        return True
    
def F16_train(config):
    callback = ReportCallback()
    model = PPO(MlpPolicy, env, verbose=1, **config)
    model.policy.optimizer = KFACOptimizer(model.policy) # <-- Plugin here!
    model.learn(total_timesteps=250000, callback=callback)
    model.save("ppo_f16")
    
analysis = tune.run(F16_train, config=config, num_samples=50, verbose=1, metric="ep_rew", mode="max", raise_on_failed_trial=False)

In [ ]:
ax = None
dfs = analysis.trial_dataframes
for d in dfs.values():
    ax = d.plot("training_iteration", "ep_rew", ax=ax, legend=False)

In [ ]:
analysis.get_best_config(metric="ep_rew", mode="max")